# LA Q3 basic characterization

includes RR, ESR, Rabi, T1

## Header that we always run

In [1]:
# set up non-QM hardware
import Labber
# connect to server 
client = Labber.connectToServer('localhost') # get list of instruments 
#instruments = client.getListOfInstrumentsString() 
#for instr in instruments: 
#    print(instr) # close connection 

# reset all QDevil channels to 0 V
QDevil = client.connectToInstrument('QDevil QDAC', dict(interface='Serial', address='3'))
for n in range(24):
    if n+1 < 10:
        QDevil.setValue("CH0" + str(n+1) + " Voltage", 0.0)
    else:
        QDevil.setValue("CH" + str(n+1) + " Voltage", 0.0)
# setting CH07 to sweet spot
DC_ss = -0.111342
QDevil.setValue("CH07 Voltage", DC_ss)

# digital attenuators
Vaunix1 = client.connectToInstrument('Painter Vaunix Lab Brick Digital Attenuator', dict(interface='USB', address='25606'))
Vaunix2 = client.connectToInstrument('Painter Vaunix Lab Brick Digital Attenuator', dict(interface='USB', address='25607'))
ROI = 42
ROO = 10
Vaunix1.setValue("Attenuation", ROI)
Vaunix2.setValue("Attenuation", ROO)

# TWPA pump
SG = client.connectToInstrument('Rohde&Schwarz RF Source', dict(interface='TCPIP', address='192.168.88.2'))
freq_TWPA = 6730.6E6;
pwr_TWPA = -11.3;
SG.setValue('Frequency', freq_TWPA)
SG.setValue('Power', pwr_TWPA)

client.close()

In [ ]:
from qm.qua import *
from qm.QuantumMachinesManager import QuantumMachinesManager
from configuration import *
import matplotlib.pyplot as plt
import numpy as np
from scipy import signal
from qm import SimulationConfig
import qdac as qdac
from scipy.io import savemat
from scipy.io import loadmat
from scipy.optimize import curve_fit
from qutip import *
import datetime 
import os
import time


qubit_index = 6 # for LA Q3

now = datetime.datetime.now()
year = now.strftime("%Y")
month = now.strftime("%m")
day = now.strftime("%d")

tPath = os.path.join(r'Z:\LabberData_DF5\QM_Data_DF5',year,month,'Data_'+month+day)
if not os.path.exists(tPath):
   os.makedirs(tPath)

2023-08-18 16:26:46,768 - qm - INFO     - Starting session: e76bac6f-eabb-423c-b5c3-01be11dc39be


In [ ]:
# fitting functions
def ESR_Shape( x, amp, sigma, x0, c ):
    return amp * sigma / ( sigma**2 + ( x - x0 )**2) + c 

def Rabi_Shape( x, amp, f, phi, c):
    return amp * np.cos(2 * np.pi * f * x + phi) + c

def exp_decay( t, A, T1, c):
    return A*np.exp(-t/T1)+c

def Phi_flux_RR(flux, c, phi0):
    # Argument for the cosine flux tuning curve
    return 2*np.pi*c*flux + phi0

def ham(flux, out, wr, Ec, Ej, c, phi0, g):
    # Hamiltonian function
    # The flux is the independent variable
    # The following are fitting parameters: wr, Ec, Ej, c, phi0, g
    # The output is the resonator frequency for all the input flux values
    # The output frequency array is in units of MHz

    N = 4 # 0-3 photons

    a = tensor(destroy(N), qeye(N)) # cavity mode
    b = tensor(qeye(N), destroy(N)) # qubit

    f_ham = []

    # Hamiltonian as a function of flux
    for k in range(np.size(flux)):
        H = wr*a.dag()*a + (np.sqrt(8*Ec*Ej*np.abs(np.cos(Phi_flux_RR(flux[0][k],c, phi0))))-Ec)*b.dag()*b - Ec/2*b.dag()*b.dag()*b*b + g*(a*b.dag()+a.dag()*b)
        w, v = np.linalg.eig(H)
        
        for n_1 in range(v.shape[1]):
            v[:,n_1] = v[:,n_1]/np.inner(v[:,n_1],v[:,n_1])

            idx_00 = np.argmax(np.abs(v[0,:])) # |0,0>
            idx_01 = np.argmax(np.abs(v[N,:])) # |1,0> photon
            idx_02 = np.argmax(np.abs(v[1,:])) # |0,1> qubit
        match out:
                case 1:
                    f_ham.append(np.abs(np.maximum(w[idx_01],w[idx_02]) - w[idx_00]))
                case 2:
                    f_ham.append(np.abs(np.minimum(w[idx_01],w[idx_02]) - w[idx_00]))
    f_ham = np.array(f_ham)
    return f_ham


def ESR_4th(arr_sz, num_steps, SS, *popt):
    # Computes ESR freqeuncies as a function of flux.
    # It creates a flux array based on two input parameters:
    # arr_sz: endpoint of array. Since it is symmetric, it is the same value
    # num_steps: number of flux points 
    # fits that to a Hamiltonian shifted by the SS so that it is centered at zero
    # and uses this to perform a fit to a fourth order polynomial with only even terms.
    # The function finally outputs the ESR frequencies as a function of flux
    x_fit_r = np.array([np.linspace(-arr_sz, arr_sz, num_steps)])
    y_fit_r = ham(x_fit_r - np.abs(SS), 2, *popt)
    Q_fit_p4_r = np.polyfit(x_fit_r[0], y_fit_r, 4)
    return Q_fit_p4_r[0]* x_fit_r**4 + Q_fit_p4_r[2] * x_fit_r**2 + Q_fit_p4_r[4]

## RR spectroscopy

In [4]:
now = datetime.datetime.now()
month = now.strftime("%m")
day = now.strftime("%d")
hour = now.strftime("%H")
minute = now.strftime("%M")

exp_name = 'RR_spec'
qubit_name = 'LA_Q3'

f_str = qubit_name + '_' + exp_name + '_' + month + day + '-' + hour + minute
file_name= f_str + '.mat'
json_name = f_str + '_state.json'

# Set-up the machine and get the config #
machine = QuAM("quam_state.json")
config = build_config(machine)

qubit_index = 6
res_if = machine.resonators[qubit_index].f_readout - machine.resonators[qubit_index].lo
n_avg = 1000
cooldown_time =20_000 # convert to ns in the wait function!

# The QUA program #
RR_freq_sweep = np.arange(-5e6, +5e6, 0.1e6)

with program() as resonator_spec:
    n = declare(int)
    n_st = declare_stream()
    df = declare(int)
    I = declare(fixed)
    Q = declare(fixed)
    I_st = declare_stream()
    Q_st = declare_stream()

    with for_(n, 0, n < n_avg, n + 1):
        save(n, n_st)
        with for_(*from_array(df, RR_freq_sweep)):
            update_frequency(machine.resonators[qubit_index].name, df + res_if)
            measure(
                "readout"*amp(1),
                machine.resonators[qubit_index].name,
                None,
                dual_demod.full("cos", "out1", "sin", "out2", I),
                dual_demod.full("minus_sin", "out1", "cos", "out2", Q),
            )
            wait(cooldown_time * u.ns, machine.resonators[qubit_index].name)
            save(I, I_st)
            save(Q, Q_st)

    with stream_processing():
        n_st.save('iteration')
        I_st.buffer(len(RR_freq_sweep)).average().save("I")
        Q_st.buffer(len(RR_freq_sweep)).average().save("Q")

#  Open Communication with the QOP  #
qmm = QuantumMachinesManager(machine.network.qop_ip, cluster_name=machine.network.cluster_name, octave=octave_config)

# Simulate or execute #
simulate = False # simulation is useful to see the sequence, especially the timing (clock cycle vs ns)

start_time = time.time()

if simulate:
    simulation_config = SimulationConfig(duration=1000)
    job = qmm.simulate(config, resonator_spec, simulation_config)
    job.get_simulated_samples().con1.plot()
else:
    qm = qmm.open_qm(config)
    job = qm.execute(resonator_spec)
    # Get results from QUA program
    res_handles = job.result_handles
    I_handles = res_handles.get("I")
    Q_handles = res_handles.get("Q")
    iteration_handles = res_handles.get("iteration")
    # Live plotting
#     fig = plt.figure()
#     plt.rcParams['figure.figsize'] = [12, 8]
#     interrupt_on_close(fig, job)  # Interrupts the job when closing the figure
    while job.result_handles.is_processing():
        # Fetch results
        iteration = res_handles.get("iteration").fetch_all()
        I = res_handles.get("I").fetch_all()
        Q = res_handles.get("Q").fetch_all()
        # progress bar
        #progress_counter(iteration, n_avg, start_time = start_time)

2023-08-18 16:27:03,874 - qm - ERROR    - Encountered connection error from QOP
Traceback (most recent call last):
  File "C:\Users\painter\AppData\Roaming\Python\Python38\site-packages\qm\api\base_api.py", line 27, in wrapped
    return func(*args, **kwargs)
  File "C:\Users\painter\AppData\Roaming\Python\Python38\site-packages\qm\api\frontend_api.py", line 79, in get_version
    response = run_async(self._stub.get_version(Empty(), timeout=self._timeout))
  File "C:\Users\painter\AppData\Roaming\Python\Python38\site-packages\qm\utils\async_utils.py", line 37, in run_async
    return create_future(coroutine).result()  # type: ignore[no-any-return]
  File "C:\Users\painter\.conda\envs\labber-qm-env\lib\concurrent\futures\_base.py", line 444, in result
    return self.__get_result()
  File "C:\Users\painter\.conda\envs\labber-qm-env\lib\concurrent\futures\_base.py", line 389, in __get_result
    raise self._exception
  File "C:\Users\painter\AppData\Roaming\Python\Python38\site-packages\

In [5]:
# Fetch results
iteration = res_handles.get("iteration").fetch_all()
I = res_handles.get("I").fetch_all()
Q = res_handles.get("Q").fetch_all()
# Convert I & Q to Volts
I = u.demod2volts(I, machine.resonators[qubit_index].readout_pulse_length)
Q = u.demod2volts(Q, machine.resonators[qubit_index].readout_pulse_length)
sigs = np.sqrt(I**2 + Q**2)
# detrend removes the linear increase of phase
phase = signal.detrend(np.unwrap(np.angle(I + 1j * Q)))
idx = np.argmin(sigs) # find minimum
print(f"IF offset to add to IF: {RR_freq_sweep[idx] / u.MHz:.6f} MHz")

# 1D spectroscopy plot
fig = plt.figure(figsize=[8, 4])
plt.title("Resonator spectroscopy")
plt.plot((machine.resonators[qubit_index].f_readout + RR_freq_sweep ) / u.MHz, sigs, ".")
plt.xlabel("Frequency [MHz]")
plt.ylabel(r"$\sqrt{I^2 + Q^2}$ [V]")
plt.axvline(x = (machine.resonators[qubit_index].f_readout + RR_freq_sweep[idx]) / u.MHz)

IF offset to add to IF: 0.100000 MHz


In [6]:
savemat(os.path.join(tPath, file_name), {"RR_freq": machine.resonators[qubit_index].f_readout + RR_freq_sweep, "sig": sigs, "sig_phase": phase, "DC_CH07": DC_ss, "ROI": ROI, "ROO": ROO, "freq_TWPA": freq_TWPA, "pwr_TWPA": pwr_TWPA})
machine._save(os.path.join(tPath, json_name), flat_data=False)

In [8]:
machine.resonators[qubit_index].f_readout += 0.0E6
machine._save("quam_state.json", flat_data=False)

## Flux sweep

In [ ]:
now = datetime.datetime.now()
month = now.strftime("%m")
day = now.strftime("%d")
hour = now.strftime("%H")
minute = now.strftime("%M")

exp_name = 'RR_spec_flux_sweep'
qubit_name = 'LA_Q3'

f_str = qubit_name + '_' + exp_name + '_' + month + day + '-' + hour + minute
file_name= f_str+'.mat'
json_name = f_str + '_state.json'


# Set-up the machine and get the config #
machine = QuAM("quam_state.json")
# machine.resonators[0].f_readout+=1.68e6
# machine.resonators[0].readout_pulse_amp=0.27
config = build_config(machine)

# Program-specific variables #
n_avg =1000  # Number of averaging loops

cooldown_time = 20_000  # Resonator cooldown time, convert to ns in the wait() function 
flux_settle_time = 250  # Flux settle time for fast flux

flux_pts = 250
#dcs = np.linspace(-0.49, 0.49, flux_pts)
dc_flux_sweep = np.arange(-9,9 + 0.1,0.1)
RR_freq_sweep = np.arange(-12.5e6, 2.5e6, 0.05e6)

res_if = machine.resonators[qubit_index].f_readout - machine.resonators[qubit_index].lo

client = Labber.connectToServer('localhost') # get list of instruments 
QDevil = client.connectToInstrument('QDevil QDAC', dict(interface='Serial', address='3'))

start_time = time.time()
for idx_dc, _ in enumerate(dc_flux_sweep):
    dc_flux = dc_flux_sweep[idx_dc]
    QDevil.setValue("CH07 Voltage", dc_flux)
# The QUA program #
    time.sleep(0.1) # To give time for the bias to settle

    with program() as resonator_spec_2D:
        n = declare(int)  # Averaging index
        df = declare(int)  # Resonator frequency
        # dc = declare(fixed)  # flux dc level|
        I = declare(fixed)
        Q = declare(fixed)
        I_st = declare_stream()
        Q_st = declare_stream()
        n_st = declare_stream()
        #wait(flux_settle_time , machine.resonators[qubit_index].name, machine.qubits[qubit_index].name)

        with for_(n, 0, n < n_avg, n + 1):
            with for_(*from_array(df, RR_freq_sweep)):
                # Update the resonator frequency
                update_frequency(machine.resonators[qubit_index].name, df + res_if)
                # with for_(*from_array(dc, dcs)):
                    # Flux sweeping
                    # set_dc_offset(machine.flux_lines[qubit_index].name, "single", dc)
                    # wait(flux_settle_time * u.ns, machine.resonators[qubit_index].name, machine.qubits[qubit_index].name)
                    # Measure the resonator
                measure(
                    "readout",
                    machine.resonators[qubit_index].name,
                    None,
                    dual_demod.full("cos", "out1", "sin", "out2", I),
                    dual_demod.full("minus_sin", "out1", "cos", "out2", Q),
                )
                # Wait for the resonator to cooldown
                wait(cooldown_time * u.ns, machine.resonators[qubit_index].name)
                # Save data to the stream processing
                save(I, I_st)
                save(Q, Q_st)
            save(n, n_st)

        with stream_processing():
            # I_st.buffer(len(dcs)).buffer(len(dfs)).average().save("I")
            # Q_st.buffer(len(dcs)).buffer(len(dfs)).average().save("Q")
            I_st.buffer(len(RR_freq_sweep)).average().save("I")
            Q_st.buffer(len(RR_freq_sweep)).average().save("Q")
            n_st.save("iteration")

    #  Open Communication with the QOP  #
    qmm = QuantumMachinesManager(machine.network.qop_ip, cluster_name=machine.network.cluster_name, octave=octave_config)

    simulation = False

    qm = qmm.open_qm(config)
    job = qm.execute(resonator_spec_2D)
    # Get results from QUA program
    results = fetching_tool(job, data_list=["I", "Q", "iteration"], mode="live")

    fig = plt.figure(1)
    plt.rcParams['figure.figsize'] = [8, 4]
    interrupt_on_close(fig, job)  #  Interrupts the job when closing the figure

    # Progress bar
    progress_counter(idx_dc, len(dc_flux_sweep), start_time=start_time)

    while results.is_processing():
        I_tmp, Q_tmp, iteration_tmp = results.fetch_all()
        I_tmp = u.demod2volts(I_tmp, machine.resonators[qubit_index].readout_pulse_length)
        Q_tmp = u.demod2volts(Q_tmp, machine.resonators[qubit_index].readout_pulse_length)

        # Live plotting
        plt.cla()
        plt.title("Resonator spectroscopy")
        plt.plot((machine.resonators[qubit_index].f_readout + RR_freq_sweep ) / u.MHz, np.sqrt(I_tmp**2 + Q_tmp**2), ".")
        plt.xlabel("Frequency [MHz]")
        plt.ylabel(r"$\sqrt{I^2 + Q^2}$ [V]")
        plt.pause(0.1)
        interrupt_on_close(fig, job)  # Interrupts the job when closing the figure

    if idx_dc == 0:
        I_2D = I_tmp
        Q_2D = Q_tmp
    else:
        I_2D = np.hstack([I_2D, I_tmp])
        Q_2D = np.hstack([Q_2D, Q_tmp])

sigs = np.sqrt(I_2D**2 + Q_2D**2)
phase = signal.detrend(np.unwrap(np.angle(I + 1j * Q)))


In [ ]:
savemat(os.path.join(tPath, file_name), {"RR_freq": machine.resonators[qubit_index].f_readout + RR_freq_sweep, "sig": sigs, "sig_phase": phase, "dc_flux_sweep": dc_flux_sweep, "ROI": ROI, "ROO": ROO, "freq_TWPA": freq_TWPA, "pwr_TWPA": pwr_TWPA})
machine._save(os.path.join(tPath, json_name), flat_data=False)

In [ ]:
client.close()

In [ ]:
# This plots the result of the RR tuning curve as well as the fitting using the Hamiltonian

# Initial guess
wr = machine.resonators[qubit_index].f_readout/1e6 # Resonator frequency
Ec = 170.080 # Capacitive energy
Ej = 30642.9 # Inductive energy 
c = 0.05 # Period in cosine function for flux
phi0 = 0.4 # Offset in cosine function for flux
g = 74.965 # RR-qubit coupling

# Load data
data = loadmat(os.path.join(tPath, file_name)) # Need to make sure this works
freq = data['RR_freq']
flux = data['dc_flux_sweep']
sig = data['sig']
phase = data['sig_phase']
sig = np.reshape(sig, (np.size(flux), np.size(freq)))
phase = np.reshape(phase, (np.size(flux), np.size(freq)))

fig = plt.figure()
plt.rcParams['figure.figsize'] = [12, 8]
plt.cla()

# 2D spectroscopy plot
plt.title("Resonator spectroscopy tuning curve")
plt.pcolormesh( np.linspace(np.min(flux), np.max(flux), np.size(flux)),np.linspace(np.min(freq), np.max(freq), np.size(freq))/1e6, sig.T, shading="nearest", cmap="seismic")
plt.xlabel("DC flux level [V]")
plt.ylabel("Frequency [MHz]")
plt.colorbar()

# Array of initial guess
guess = [wr, Ec, Ej, c, phi0, g]

# Frequency output from hamiltonian function using initial guess
freq_fit_guess = ham(flux, 1, *guess)

# Plot of initial guess
plt.plot(flux.T, freq_fit_guess)

# Determine minima of signal to use as ydata for fitting
minimas = []
min_sig = []
for i in range(np.size(flux)):
    minimas.append(freq[0][np.argmin(sig[i])])
minimas = np.array(minimas)
min_sig = np.array(min_sig)

## Plot minima of signal
# plt.scatter(flux.T, minimas/1e6)

minimas_mhz = minimas/1e6

# Fit data from minima of signal to Hamiltonian function
# Initial guess is given by p0
# Bounds are necessary. The order is bounds = ((min_wr, min_Ec, min_Ej, min_c, min_phi0, min_g), (max_wr, max_Ec, max_Ej, max_c, max_phi0, max_g))
popt, _ = curve_fit(lambda flux, *guess: ham(flux, 1, *guess), xdata = flux, ydata = minimas_mhz, p0=guess, check_finite = "true", bounds = ((wr-200, Ec-100, Ej-20000, 0.0001, -4, g-50),(wr+200, Ec+100 ,Ej+ 20000, 4, 4, g+50)))

# This is the frequency output from the fitted data
freq_fit = ham(flux, 1, *popt)

# Plot the fitted data
plt.plot(flux.T, freq_fit)

plt.legend(["guess", "fit"])
plt.show()

In [ ]:
# Fitted ESR plot
freq_fit_qubit = ham(flux, 2, *popt)

plt.title("Qubit spectroscopy tuning curve")
plt.plot(flux.T, freq_fit_qubit)
plt.xlabel("DC flux level [V]")
plt.ylabel("Frequency [MHz]")

# SS from fitting to Hamiltonian parameters
SS = (-popt[4])/(2*np.pi*popt[3])
# Get the sweet spot based from fitted parameters to Hamiltonian
# Only cosine term matters for this
print('SS at: %.6f\n' % SS) 

# Fit ESR to a fourth order polynomial centered at zero (SS was shifted)
num_steps = 199
arr_sz = 0.99 # Choose half of the size we will sweep around SS
# Shift SS to be at 0 V, as ff is relative to the SS
ESR_freqs = ESR_4th(arr_sz, num_steps, SS, *popt) # get array of qubit freqeuncies as a function of flux with SS at 0V
ESR_freq_max = np.max(ESR_freqs[0])
ESR_freq_min = np.min(ESR_freqs[0])

## ESR sweet spot

In [10]:
# set DC flux to sweet spot
client = Labber.connectToServer('localhost') # get list of instruments 
QDevil = client.connectToInstrument('QDevil QDAC', dict(interface='Serial', address='3'))

DC_ss = SS # -0.111342
QDevil.setValue("CH07 Voltage", DC_ss)
client.close()

In [11]:
now = datetime.datetime.now()
month = now.strftime("%m")
day = now.strftime("%d")
hour = now.strftime("%H")
minute = now.strftime("%M")

exp_name = 'ESR'
qubit_name = 'LA_Q3'

f_str = qubit_name + '_' + exp_name + '_' + month + day + '-' + hour + minute
file_name= f_str+'.mat'
json_name = f_str + '_state.json'

# Set-up the machine and get the config #
machine = QuAM("quam_state.json")
config = build_config(machine)

# Program-specific variables #
n_avg = 2000  # Number of averaging loops
cooldown_time = 20_000  # convert to ns in the wait() function
t = 80 * u.ns  # Qubit pulse length
freq_sweep = np.arange(-40e6, + 40e6, 1.0e6)

qubit_index = 6
qb_if = machine.qubits[qubit_index].f_01 - machine.qubits[qubit_index].lo

# The QUA program #
with program() as qubit_spec:
    n = declare(int)  # Averaging index
    df = declare(int)  # Resonator frequency
    I = declare(fixed)
    Q = declare(fixed)
    I_st = declare_stream()
    Q_st = declare_stream()
    n_st = declare_stream()

    # Adjust the flux line if needed
    # set_dc_offset("flux_line", "single", 0.0)
    with for_(n, 0, n < n_avg, n + 1):
        with for_(*from_array(df, freq_sweep)):
            # Update the qubit frequency
            update_frequency(machine.qubits[qubit_index].name, df + qb_if)
            # Play a saturation pulse on the qubit
            # play("cw" * amp(0.05), machine.qubits[qubit_index].name, duration = t)
            play("x180" * amp(1.0), machine.qubits[qubit_index].name)
            align(machine.qubits[qubit_index].name, machine.resonators[qubit_index].name)
            # Measure the resonator
            measure(
                "readout",
                machine.resonators[qubit_index].name,
                None,
                dual_demod.full("cos", "out1", "sin", "out2", I),
                dual_demod.full("minus_sin", "out1", "cos", "out2", Q),
            )
            # Wait for the resonator to cooldown
            wait(cooldown_time * u.ns, machine.resonators[qubit_index].name)
            # Save data to the stream processing
            save(I, I_st)
            save(Q, Q_st)
        save(n, n_st)

    with stream_processing():
        I_st.buffer(len(freq_sweep)).average().save("I")
        Q_st.buffer(len(freq_sweep)).average().save("Q")
        n_st.save("iteration")

#  Open Communication with the QOP  #
qmm = QuantumMachinesManager(machine.network.qop_ip, cluster_name=machine.network.cluster_name, octave=octave_config)
simulation = False

start_time = time.time()
if simulation:
    simulation_config = SimulationConfig(
        duration=8000, simulation_interface=LoopbackInterface([("con1", 3, "con1", 1)])
    )
    job = qmm.simulate(config, qubit_spec, simulation_config)
    job.get_simulated_samples().con1.plot()
else:
    qm = qmm.open_qm(config)
    job = qm.execute(qubit_spec)
    # Get results from QUA program
    res_handles = job.result_handles
    I_handles = res_handles.get("I")
    Q_handles = res_handles.get("Q")
    iteration_handles = res_handles.get("iteration")
    # Live plotting
#     fig = plt.figure()
#     plt.rcParams['figure.figsize'] = [12, 8]
#     interrupt_on_close(fig, job)  # Interrupts the job when closing the figure
    while job.result_handles.is_processing():
        # Fetch results
        iteration = res_handles.get("iteration").fetch_all()
        I = res_handles.get("I").fetch_all()
        Q = res_handles.get("Q").fetch_all()
        #progress_counter(iteration, n_avg, start_time = start_time)

2023-08-18 16:28:36,450 - qm - ERROR    - Encountered connection error from QOP
Traceback (most recent call last):
  File "C:\Users\painter\AppData\Roaming\Python\Python38\site-packages\qm\api\base_api.py", line 27, in wrapped
    return func(*args, **kwargs)
  File "C:\Users\painter\AppData\Roaming\Python\Python38\site-packages\qm\api\frontend_api.py", line 79, in get_version
    response = run_async(self._stub.get_version(Empty(), timeout=self._timeout))
  File "C:\Users\painter\AppData\Roaming\Python\Python38\site-packages\qm\utils\async_utils.py", line 37, in run_async
    return create_future(coroutine).result()  # type: ignore[no-any-return]
  File "C:\Users\painter\.conda\envs\labber-qm-env\lib\concurrent\futures\_base.py", line 444, in result
    return self.__get_result()
  File "C:\Users\painter\.conda\envs\labber-qm-env\lib\concurrent\futures\_base.py", line 389, in __get_result
    raise self._exception
  File "C:\Users\painter\AppData\Roaming\Python\Python38\site-packages\

In [12]:
# Fetch results
iteration = res_handles.get("iteration").fetch_all()
I = res_handles.get("I").fetch_all()
Q = res_handles.get("Q").fetch_all()
# Convert I & Q to Volts
I = u.demod2volts(I, machine.resonators[qubit_index].readout_pulse_length)
Q = u.demod2volts(Q, machine.resonators[qubit_index].readout_pulse_length)
sigs = np.sqrt(I**2 + Q**2)
phase = signal.detrend(np.unwrap(np.angle(I + 1j * Q)))
freqs = machine.qubits[qubit_index].f_01 + freq_sweep

savemat(os.path.join(tPath, file_name), {"ESR_freq": machine.qubits[qubit_index].f_01 + freq_sweep, "sig": sigs, "sig_phase": phase, "DC_flux": DC_ss, "ROI": ROI, "ROO": ROO, "freq_TWPA": freq_TWPA, "pwr_TWPA": pwr_TWPA})
machine._save(os.path.join(tPath, json_name), flat_data=False)

In [13]:
# fit the data
try:
    param, dp_cov=curve_fit(ESR_Shape,freqs,sigs,p0=[max(sigs)-min(sigs),8E6,freqs[np.argmax(sigs)],min(sigs)] )
    d_ESR_freq = round(param[2] * 100)/100
    print('Updated f_01 (df) [MHz]:', param[2], (param[2] - machine.qubits[qubit_index].f_01) / u.MHz)
except:
    pass

# 1D spectroscopy plot
fig = plt.figure(figsize=[14, 6])
plt.plot(freqs / u.MHz, sigs, "b.")
plt.plot(freqs / u.MHz, ESR_Shape(freqs, param[0],param[1],param[2],param[3]), 'r')
plt.show()
plt.title("Qubit spectroscopy amplitude")

plt.xlabel("Frequency [MHz]")
plt.ylabel(r"$\sqrt{I^2 + Q^2}$ [V]")
plt.axvline(x=machine.qubits[qubit_index].f_01/u.MHz)

Updated f_01 (df) [MHz]: 5443042924.143985 1.0429241439847947


In [14]:
machine.qubits[qubit_index].f_01 = 5443042924
machine._save("quam_state.json", flat_data=False)

## Rabi

In [16]:
now = datetime.datetime.now()
month = now.strftime("%m")
day = now.strftime("%d")
hour = now.strftime("%H")
minute = now.strftime("%M")

exp_name = 'Rabi'
qubit_name = 'LA_Q3'

f_str = qubit_name + '_' + exp_name + '_' + month + day + '-' + hour + minute
file_name= f_str+'.mat'
json_name = f_str + '_state.json'

# Set-up the machine and get the config #
machine = QuAM("quam_state.json")
# machine.qubits[0].pi_amp=0.109
# machine.qubits[0].pi_length=80
config = build_config(machine)

# The QUA program #
n_avg = 2_000
cooldown_time = 20_000 # ns, set in wait() function
amps = np.arange(0.1, 1.99, 0.01)
err_amp = 1  # Number of played qubit pulses for getting a better estimate of the pi amplitude
qubit_index = 6

with program() as power_rabi:
    n = declare(int)
    n_st = declare_stream()
    a = declare(fixed)
    I = declare(fixed)
    Q = declare(fixed)
    I_st = declare_stream()
    Q_st = declare_stream()

    with for_(n, 0, n < n_avg, n + 1):
        with for_(*from_array(a, amps)):
            # Loop for error amplification (perform many qubit pulses)
            for i in range(err_amp):
                play("x180" * amp(a), machine.qubits[qubit_index].name)
            align(machine.qubits[qubit_index].name, machine.resonators[qubit_index].name)
            measure(
                "readout",
                machine.resonators[qubit_index].name,
                None,
                dual_demod.full("cos", "out1", "sin", "out2", I),
                dual_demod.full("minus_sin", "out1", "cos", "out2", Q),
            )
            save(I, I_st)
            save(Q, Q_st)
            wait(cooldown_time * u.ns, machine.resonators[qubit_index].name)
        save(n, n_st)

    with stream_processing():
        I_st.buffer(len(amps)).average().save("I")
        Q_st.buffer(len(amps)).average().save("Q")
        n_st.save("iteration")

#  Open Communication with the QOP  #
qmm = QuantumMachinesManager(machine.network.qop_ip, cluster_name=machine.network.cluster_name, octave=octave_config)

simulate = False
if simulate:
    simulation_config = SimulationConfig(duration=1000)  # in clock cycles
    job = qmm.simulate(config, power_rabi, simulation_config)
    job.get_simulated_samples().con1.plot()
else:
    qm = qmm.open_qm(config)
    job = qm.execute(power_rabi)
    # Get results from QUA program
    results = fetching_tool(job, data_list=["I", "Q", "iteration"], mode="live")

    while results.is_processing():
        # Fetch results
        I, Q, iteration = results.fetch_all()
        # Progress bar
        progress_counter(iteration, n_avg, start_time=results.get_start_time())

2023-08-18 16:30:03,476 - qm - ERROR    - Encountered connection error from QOP
Traceback (most recent call last):
  File "C:\Users\painter\AppData\Roaming\Python\Python38\site-packages\qm\api\base_api.py", line 27, in wrapped
    return func(*args, **kwargs)
  File "C:\Users\painter\AppData\Roaming\Python\Python38\site-packages\qm\api\frontend_api.py", line 79, in get_version
    response = run_async(self._stub.get_version(Empty(), timeout=self._timeout))
  File "C:\Users\painter\AppData\Roaming\Python\Python38\site-packages\qm\utils\async_utils.py", line 37, in run_async
    return create_future(coroutine).result()  # type: ignore[no-any-return]
  File "C:\Users\painter\.conda\envs\labber-qm-env\lib\concurrent\futures\_base.py", line 444, in result
    return self.__get_result()
  File "C:\Users\painter\.conda\envs\labber-qm-env\lib\concurrent\futures\_base.py", line 389, in __get_result
    raise self._exception
  File "C:\Users\painter\AppData\Roaming\Python\Python38\site-packages\

In [17]:
I, Q, iteration = results.fetch_all()
# Convert I & Q to Volts
I = u.demod2volts(I, machine.resonators[qubit_index].readout_pulse_length)
Q = u.demod2volts(Q, machine.resonators[qubit_index].readout_pulse_length)
sigs = np.sqrt(I**2 + Q**2)
phase = signal.detrend(np.unwrap(np.angle(I + 1j * Q)))

savemat(os.path.join(tPath, file_name), {"Rabi_amp": machine.qubits[qubit_index].pi_amp * amps, "sig": sigs, "sig_phase": phase, "DC_flux": DC_ss, "ROI": ROI, "ROO": ROO, "freq_TWPA": freq_TWPA, "pwr_TWPA": pwr_TWPA})
machine._save(os.path.join(tPath, json_name), flat_data=False)

In [18]:
# fit the data
try:
    param, dp_cov=curve_fit(Rabi_Shape,amps,sigs,p0=[(max(sigs)-min(sigs))/2,1.2,0,(max(sigs)+min(sigs))/2] )
    amp_pi = round(((0.5-param[2]/2/np.pi)/param[1]) * 10000)/10000
    print('Updated amp_pi [V]:', amp_pi)
except:
    pass

# 1D spectroscopy plot
fig = plt.figure(figsize=[14, 6])
plt.plot(amps, sigs, "b.")
plt.plot(amps, Rabi_Shape(amps, param[0],param[1],param[2],param[3]), 'r')
plt.show()
plt.title("Rabi pulse amplitude [V]")

plt.xlabel("Amplitude [a.u.]")
plt.ylabel(r"$\sqrt{I^2 + Q^2}$ [V]")
plt.axvline(x=amp_pi)

Updated amp_pi [V]: 0.0068


In [ ]:
machine.qubits[qubit_index].pi_amp = machine.qubits[qubit_index].pi_amp * amp_pi
machine._save("quam_state.json", flat_data=False)

## T1

In [25]:
now = datetime.datetime.now()
month = now.strftime("%m")
day = now.strftime("%d")
hour = now.strftime("%H")
minute = now.strftime("%M")

exp_name = 'T1'
qubit_name = 'LA_Q3'

f_str = qubit_name + '_' + exp_name + '_' + month + day + '-' + hour + minute
file_name= f_str+'.mat'
json_name = f_str + '_state.json'

machine = QuAM("quam_state.json")
config = build_config(machine)

# The QUA program #
tau_min = 100 # clock cycle
tau_max = 5_000 # clock cycle
d_tau = 100 # clock cycle
tau_sweep = np.arange(tau_min, tau_max + 0.1, d_tau)  # + 0.1 to add t_max to taus
qubit_index = 6

n_avg = 10000
cooldown_time = 20_000 # ns

with program() as T1:
    n = declare(int)
    n_st = declare_stream()
    I = declare(fixed)
    I_st = declare_stream()
    Q = declare(fixed)
    Q_st = declare_stream()
    tau = declare(int)

    with for_(n, 0, n < n_avg, n + 1):
        with for_(*from_array(tau, tau_sweep)):
            play("x180", machine.qubits[qubit_index].name)
            wait(tau, machine.qubits[qubit_index].name)
            align(machine.qubits[qubit_index].name, machine.resonators[qubit_index].name)
            measure(
                "readout",
                machine.resonators[qubit_index].name,
                None,
                dual_demod.full("rotated_cos", "out1", "rotated_sin", "out2", I),
                dual_demod.full("rotated_minus_sin", "out1", "rotated_cos", "out2", Q),
            )
            save(I, I_st)
            save(Q, Q_st)
            wait(cooldown_time * u.ns, machine.resonators[qubit_index].name)
        save(n, n_st)

    with stream_processing():
        I_st.buffer(len(tau_sweep)).average().save("I")
        Q_st.buffer(len(tau_sweep)).average().save("Q")
        n_st.save("iteration")

#  Open Communication with the QOP  #
qmm = QuantumMachinesManager(machine.network.qop_ip, cluster_name=machine.network.cluster_name, octave=octave_config)

# Simulate or execute #
simulate = False
if simulate:
    simulation_config = SimulationConfig(duration=20_000)  # in clock cycles
    job = qmm.simulate(config, T1, simulation_config)
    job.get_simulated_samples().con1.plot()

else:
    print(datetime.datetime.now(),2)
    qm = qmm.open_qm(config)
    job = qm.execute(T1)
    
    # Get results from QUA program
    results = fetching_tool(job, data_list=["I", "Q", "iteration"], mode="live")
    # Live plotting
#     fig = plt.figure()
#     plt.rcParams['figure.figsize'] = [8, 6]
#     interrupt_on_close(fig, job)  # Interrupts the job when closing the figure
    while results.is_processing():
        # Fetch results
        I, Q, iteration = results.fetch_all()
        # Progress bar
        progress_counter(iteration, n_avg, start_time=results.get_start_time())

2023-08-18 16:33:13,152 - qm - ERROR    - Encountered connection error from QOP
Traceback (most recent call last):
  File "C:\Users\painter\AppData\Roaming\Python\Python38\site-packages\qm\api\base_api.py", line 27, in wrapped
    return func(*args, **kwargs)
  File "C:\Users\painter\AppData\Roaming\Python\Python38\site-packages\qm\api\frontend_api.py", line 79, in get_version
    response = run_async(self._stub.get_version(Empty(), timeout=self._timeout))
  File "C:\Users\painter\AppData\Roaming\Python\Python38\site-packages\qm\utils\async_utils.py", line 37, in run_async
    return create_future(coroutine).result()  # type: ignore[no-any-return]
  File "C:\Users\painter\.conda\envs\labber-qm-env\lib\concurrent\futures\_base.py", line 444, in result
    return self.__get_result()
  File "C:\Users\painter\.conda\envs\labber-qm-env\lib\concurrent\futures\_base.py", line 389, in __get_result
    raise self._exception
  File "C:\Users\painter\AppData\Roaming\Python\Python38\site-packages\

In [26]:
# Fetch results
I, Q, iteration = results.fetch_all()
# Convert I & Q to Volts
I = u.demod2volts(I, machine.resonators[qubit_index].readout_pulse_length)
Q = u.demod2volts(Q, machine.resonators[qubit_index].readout_pulse_length)
sigs = np.sqrt(I**2 + Q**2)
phase = signal.detrend(np.unwrap(np.angle(I + 1j * Q)))
taus = tau_sweep * 4.0 * u.ns # convert clock cycle to ns

savemat(os.path.join(tPath, file_name), {"tau": taus, "sig": sigs, "sig_phase": phase, "DC_flux": DC_ss, "ROI": ROI, "ROO": ROO, "freq_TWPA": freq_TWPA, "pwr_TWPA": pwr_TWPA})
machine._save(os.path.join(tPath, json_name), flat_data=False)

In [28]:
try:
    param, _ = curve_fit(exp_decay,taus,sigs,p0=[max(sigs)-min(sigs),4E3,min(sigs)] )
    q_T1 = round(param[1])
    print('Qubit T1 [us]:', q_T1 /u.us)
except:
    pass

# 1D spectroscopy plot
fig = plt.figure(figsize=[14, 6])
plt.plot(taus /u.us, sigs, "b.")
plt.plot(taus / u.us, exp_decay(taus, param[0],param[1],param[2]), 'r')
plt.show()
plt.title("Qubit T1")
plt.xlabel("tau [us]")
plt.ylabel("$\sqrt{I^2 + Q^2}$ [V]")

Qubit T1 [us]: 3.454


Text(0, 0.5, '$\\sqrt{I^2 + Q^2}$ [V]')

In [29]:
machine.qubits[qubit_index].T1 = q_T1
machine._save("quam_state.json", flat_data=False)

## ESR with flux (coarse)
### Sweep the qubit frequency as a function of flux using ff (OPX) and optimize RR frequency from Hamiltonian fit during RR spec (improve readout SNR)
### Use the results to fit to a fourth order polynomial and create the fine ESR flux sweep
### This part of the code has not been tested



now = datetime.datetime.now()
month = now.strftime("%m")
day = now.strftime("%d")
hour = now.strftime("%H")
minute = now.strftime("%M")

exp_name = 'ESR_spec_flux_sweep'
qubit_name = 'LA_Q3'

f_str = qubit_name + '_' + exp_name + '_' + month + day + '-' + hour + minute
file_name= f_str+'.mat'
json_name = f_str + '_state.json'


# Set-up the machine and get the config #
machine = QuAM("quam_state.json")
config = build_config(machine)

# Program-specific variables #
n_avg =1000  # Number of averaging loops

cooldown_time = 20_000  # Resonator cooldown time, convert to ns in the wait() function 
flux_settle_time = 250  # Flux settle time for fast flux from OP

# Sweeping ranges for flux and frequency 
flux_sweep = np.arange(-0.05,0.05, 0.001)
qubit_freq_sweep = np.arange(-10e6, 10e6, 0.5e6)

# IF frequencies
qb_if = [machine.qubits[qubit_index].f_01 - machine.qubits[qubit_index].lo

fitted_curve_RR = ham(flux_sweep, 1, *popt) # Need RR spec with flux script fitting
fitted_curve_RR = fitted_curve_RR*1e6
fitted_curve_RR = fitted_curve_RR.astype(int)

res_if = machine.resonators[qubit_index].f_readout - machine.resonators[qubit_index].lo

client = Labber.connectToServer('localhost') # get list of instruments 
# QDevil = client.connectToInstrument('QDevil QDAC', dict(interface='Serial', address='3'))

start_time = time.time()


## The QUA program #
# time.sleep(0.1) # To give time for the bias to settle

with program() as qubit_spec_2D:
    n = declare(int)  # Averaging index
    df = declare(int)  # Qubit frequency
    dc = declare(fixed)  # ff
    I = declare(fixed)
    Q = declare(fixed)
    res_freqs = declare(int, value = fitted_curve_RR.tolist())
    index = declare(int, value = 0)
    I_st = declare_stream()
    Q_st = declare_stream()
    n_st = declare_stream()

    with for_(n, 0, n < n_avg, n + 1):
        with for_(*from_array(df, qubit_freq_sweep)):
            # Update the qubit frequency
            update_frequency(machine.qubits[qubit_index].name, df + qb_if)
            assign(index, 0)
            with for_(*from_array(dc, flux_sweep)):
                # Flux sweeping
                update_frequency(machine.resonators[qubit_index].name, res_freqs[index] + res_if)
                set_dc_offset(machine.flux_lines[qubit_index].name, "single", dc)
                wait(flux_settle_time * u.ns, machine.resonators[qubit_index].name, machine.qubits[qubit_index].name)
                # Play a pi pulse on the qubit
                play("x180" * amp(1), machine.qubits[qubit_index].name)
                # play("cw" * amp(.1), machine.qubits[qubit_index].name, duration=t*u.ns)
                align(machine.qubits[qubit_index].name, machine.resonators[qubit_index].name)
                # Measure the resonator
                measure(
                    "readout",
                    machine.resonators[qubit_index].name,
                    None,
                    dual_demod.full("cos", "out1", "sin", "out2", I),
                    dual_demod.full("minus_sin", "out1", "cos", "out2", Q),
                )
                # Wait for the resonator to cooldown
                wait(cooldown_time * u.ns, machine.resonators[qubit_index].name)
                # Save data to the stream processing
                save(I, I_st)
                save(Q, Q_st)
                assign(index, index + 1)
        save(n, n_st)

    with stream_processing():
        I_st.buffer(len(flux_sweep)).buffer(len(qubit_freq_sweep)).average().save("I")
        Q_st.buffer(len(flux_sweep)).buffer(len(qubit_freq_sweep)).average().save("Q")
        # I_st.buffer(len(qubit_freq_sweep)).average().save("I")
        # Q_st.buffer(len(qubit_freq_sweep)).average().save("Q")
        n_st.save("iteration")

#  Open Communication with the QOP  #
qmm = QuantumMachinesManager(machine.network.qop_ip, cluster_name=machine.network.cluster_name, octave=octave_config)

simulation = False

qm = qmm.open_qm(config)
job = qm.execute(qubit_spec_2D)
# Get results from QUA program
results = fetching_tool(job, data_list=["I", "Q", "iteration"], mode="live")

fig = plt.figure(1)
plt.rcParams['figure.figsize'] = [12, 8]
interrupt_on_close(fig, job)  #  Interrupts the job when closing the figure

# Progress bar
progress_counter(idx_dc, len(dc_flux_sweep), start_time=start_time)

while results.is_processing():
    I, Q, iteration = results.fetch_all()
    I = u.demod2volts(I, machine.resonators[qubit_index].readout_pulse_length)
    Q = u.demod2volts(Q, machine.resonators[qubit_index].readout_pulse_length)

    # Live plotting
    plt.cla()
    plt.title("Qubit spectroscopy")
    plt.pcolor(flux_sweep, (machine.qubits[qubit_index].f_01 + qubit_freq_sweep ) / u.MHz, np.sqrt(I**2 + Q**2),cmap="seismic")
    plt.ylabel("Frequency [MHz]")
    plt.xlabel("Flux level [V]")
    plt.pause(0.1)
    interrupt_on_close(fig, job)  # Interrupts the job when closing the figure


sigs = np.sqrt(I**2 + Q**2)
phase = signal.detrend(np.unwrap(np.angle(I + 1j * Q)))

In [ ]:
savemat(os.path.join(tPath, file_name), {"ESR_freq": machine.qubits[qubit_index].f_01 + qubit_freq_sweep, "sig": sigs, "sig_phase": phase, "fast_flux_sweep": flux_sweep, "ROI": ROI, "ROO": ROO, "freq_TWPA": freq_TWPA, "pwr_TWPA": pwr_TWPA})
machine._save(os.path.join(tPath, json_name), flat_data=False)

In [ ]:
# Find the minima in the signal and fit it to a fourth order polynomial function
# Load data
data = loadmat(os.path.join(tPath, file_name)) # Need to make sure this works
freq = data['ESR_freq']
flux = data['fast_flux_sweep']
sig = data['sig']
phase = data['sig_phase']
sig = np.reshape(sig, (np.size(flux), np.size(freq)))
phase = np.reshape(phase, (np.size(flux), np.size(freq)))

fig = plt.figure()
plt.rcParams['figure.figsize'] = [12, 8]
plt.cla()

# 2D spectroscopy plot
plt.title("Resonator spectroscopy tuning curve")
plt.pcolormesh( np.linspace(np.min(flux), np.max(flux), np.size(flux)),np.linspace(np.min(freq), np.max(freq), np.size(freq))/1e6, sig.T, shading="nearest", cmap="seismic")
plt.xlabel("DC flux level [V]")
plt.ylabel("Frequency [MHz]")
plt.colorbar()

# Determine minima of signal to use as ydata for fitting
minimas = []
min_sig = []
for i in range(np.size(flux)):
    minimas.append(freq[0][np.argmin(sig[i])])
minimas = np.array(minimas)
min_sig = np.array(min_sig)
minimas_mhz = minimas/1e6

# Curve fit to fourth order polynomial
Q_fit_p4_q = np.polyfit(flux[0], minimas_mhz, 4)
plt.scatter(flux[0], np.polyval(Q_fit_p4_q, flux[0]).T)

qubit_freqs_4th = np.polyval(Q_fit_p4_q, flux[0])

In [ ]:
<!-- fitted_curve_Q = ham(flux_sweep, *popt) # Need RR spec with flux script fitting
fitted_curve_Q = fitted_curve_RR*1e6
fitted_curve_Q = fitted_curve_RR.astype(int) -->